# Build a summarizer using the SpaCy-LLM module

## Imports

In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

## Calculate the word frequencies
First, we need to process the text, tokenize it and calculate the frequency of all of the words.

Calculating word frequency is crucial in text summarization because it helps identify the most important words and themes, allowing the summarizer to focus on and include the most relevant sentences in the summary.

In [2]:
def calculate_word_frequencies(doc):
    freq_of_word = {}
    for word in doc:
        if word.text.lower() not in STOP_WORDS and word.text.lower() not in punctuation:
            if word.text not in freq_of_word.keys():
                freq_of_word[word.text] = 1
            else:
                freq_of_word[word.text] += 1
    return freq_of_word

## Normalize word frequencies

This function gets rid of the bias for longer texts, or words that are used very frequently. The importance of a word is judged relative to the most frequent word, rather than by its absolute frequency count. This way the summarization is more accurate.

In [3]:
def normalize_frequencies(freq_of_word):
    max_freq = max(freq_of_word.values())
    for word in freq_of_word:
        freq_of_word[word] = freq_of_word[word] / max_freq
    return freq_of_word

## Score sentences based on the frequency of the words

This process involves evaluating each sentence in a document to determine its relevance and importance to the overall content, using the frequencies of the words it contains as a primary metric.

In [4]:
def score_sentences(doc, freq_of_word):
    sent_scores = {}
    for sent in doc.sents:
        for word in sent:
            if word.text.lower() in freq_of_word:
                if sent not in sent_scores:
                    sent_scores[sent] = freq_of_word[word.text.lower()]
                else:
                    sent_scores[sent] += freq_of_word[word.text.lower()]
    return sent_scores

## Generate summary

In [5]:
def generate_summary(doc, sent_scores, percentage=0.1):
    len_tokens = int(len(list(doc.sents)) * percentage)
    summary = nlargest(n=len_tokens, iterable=sent_scores, key=sent_scores.get)
    final_summary = [word.text for word in summary]
    return " ".join(final_summary)

## Summarize the input text

In [9]:
def text_summarizer(text, percentage):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    freq_of_word = calculate_word_frequencies(doc)
    freq_of_word = normalize_frequencies(freq_of_word)
    sent_scores = score_sentences(doc, freq_of_word)
    summary = generate_summary(doc, sent_scores, percentage)
    return summary

## Input text

In [10]:
text = """I often dream about the Dolphin Hotel. In these dreams, I'm there, implicated in some kind of ongoing circumstance. All indications are that I belong to this dream continuity. The Dolphin Hotel is distorted, much too narrow. It seems more like a long, covered bridge. A bridge stretching endlessly through time. And there I am, in the middle of it. Someone else is there too, crying. The hotel envelops me. I can feel its pulse, its heat. In dreams, I am part of the hotel. I wake up, but where? I don't just think this, I actually voice the question to myself: «Where am I?» As if I didn't know: I'm here. In my life. A feature of the world that is my existence. Not that I particularly recall ever having approved these matters, this condition, this state of affairs in which I feature. There might be a woman sleeping next to me. More often, I'm alone. Just me and the expressway that runs right next to my apartment and, bedside, a glass (five millimeters of whiskey still in it) and the malicious — no, make that indifferent—dusty morning light. Sometimes it's raining. If it is, I'll just stay in bed. And if there's whiskey still left in the glass, I'll drink it. And I'll look at the raindrops dripping from the eaves, and I'll think about the Dolphin Hotel. Maybe I'll stretch, nice and slow. Enough for me to be sure I'm myself and not part of something else. Yet I'll remember the feel of the dream. So much that I swear I can reach out and touch it, and the whole of that something that includes me will move. If I strain my ears, I can hear the slow, cautious sequence of play take place, like droplets in an intricate water puzzle falling, step upon step, one after the other. I listen carefully. That's when I hear someone softly, almost imperceptibly, weeping. A sobbing from somewhere in the darkness. Someone is crying for me. The Dolphin Hotel is a real hotel. It actually exists in a so-so section of Sapporo. Once, a few years back, I spent a week there. No, let me get that straight. How many years ago was it? Four. Or more precisely, four and a half. I was still in my twenties. I checked into the Dolphin Hotel with a woman I was living with. She'd chosen the place. This is where we're staying, was what she said. If it hadn't been for her, I doubt I'd ever have set foot in the place. It was a tiny dump of a hotel. In the whole time we were there, I don't know if we saw another paying customer. There were a couple of characters milling around the lobby, but who knows if they were staying there? A few keys were always missing from the board behind the front desk, so I guess there were other hotel guests. Though not too many. I mean, really, you hang out a hotel sign somewhere in a major city, put a phone number in the business listings, it stands to reason you're not going to go entirely without cus-tomers. But granting there were other customers besides our-selves, they were awfully quiet. We never heard a sound from them, hardly saw a sign of their presence—with the exception of the arrangement of the keys on the board that changed slightly each day. Were they like shadows creeping along the walls of the corridors, holding their breath? Occasionally we'd hear the dull rattling of the elevator, but when it stopped the oppressive silence bore down once more. A mysterious hotel. What it reminded me of was a biological dead end. A ge-netic retrogression. A freak accident of nature that stranded some organism up the wrong path without a way back. Evo-lutionary vector eliminated, orphaned life-form left cowering behind the curtain of history, in The Land That Time Forgot. And through no fault of anyone. No one to blame, no one to save it. The hotel should never have been built where it was. That was the first mistake, and everything got worse from there. Like a button on a shirt buttoned wrong, every attempt to correct things led to yet another fine—not to say elegant— mess. No detail seemed right. Look at anything in the place and you'd find yourself tilting your head a few degrees. Not enough to cause you any real harm, nor enough to seem par-ticularly odd. Who knows? You might get used to this slant on things (but if you did, you'd never be able to view the world again without holding your head out of true)."""

## Select 20% sentences with maximum score as a final summary

In [12]:
final_summary = text_summarizer(text, 0.2)

## Summary of the text

In [13]:
print("#"*50)
print("Lenght of the initial text:", len(text))
print("Length of summarized text:",len(final_summary))
print("#"*50)
print()
print(final_summary)

##################################################
Lenght of the initial text: 4271
Length of summarized text: 1538
##################################################

Just me and the expressway that runs right next to my apartment and, bedside, a glass (five millimeters of whiskey still in it) and the malicious — no, make that indifferent—dusty morning light. If I strain my ears, I can hear the slow, cautious sequence of play take place, like droplets in an intricate water puzzle falling, step upon step, one after the other. Like a button on a shirt buttoned wrong, every attempt to correct things led to yet another fine—not to say elegant— mess. I mean, really, you hang out a hotel sign somewhere in a major city, put a phone number in the business listings, it stands to reason you're not going to go entirely without cus-tomers. We never heard a sound from them, hardly saw a sign of their presence—with the exception of the arrangement of the keys on the board that changed slightly each